In [2]:
!pip install pandas numpy 

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 1.7 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/11.1 MB 1.7 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/11.1 MB 1.7 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.1 MB 1.8 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.1 MB 2.1 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.1 MB 2.4 MB/s eta 0:00:04
   -------------- ------------------------- 3.9/11.1 MB 2.6 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.1 MB 3.0 MB/s eta 0:00:02
   ------------------------- -------------- 7.1/11.1 MB 3.6 MB/s eta 0:00:02
   -------------------------------- ------- 8.9/11.1 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 4.7 MB/s eta 0:00:01
   ---------

In [3]:
import pandas as pd


In [ ]:
# detailed_trajectory = pd.read_pickle("./data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")
road_df=pd.read_csv("./LaDe/road-network/roads.csv",sep="\t")
road_df = road_df[road_df["city"]=="杭州市"]      # place in Hangzhou

In [ ]:
road_df.columns

# osm_id = openstreetmap_id 
# 'code' = osm internal code schema 
# 'fclass', =  functional category of road -  motorway, primary, secondary, residential
#  'name', = official name availble in osm
#  'ref'   = Reference code (like highway numbers, e.g., “G60” for a national expressway). ex. S102 
#  'oneway', = Indicates if the road is one-way (yes/no). Critical for routing algorithms.
#  'maxspeed'= Maximum legal speed limit on the road segment (km/h).
#  'layer' = Vertical layer of the road (used when roads overlap, e.g., flyovers vs. tunnels) --> 
#  'bridge'= Boolean flag (yes/no) indicating if the segment is a bridge.
#  'tunnel'= Boolean flag (yes/no) indicating if the segment is a tunnel.
#  'city', = The city to which the road belongs (Shanghai, Hangzhou, Chongqing, Jilin, Yantai).
# 'geometry' = The polyline coordinates (latitude/longitude pairs) describing the exact shape of the road segment.

Index(['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed',
       'layer', 'bridge', 'tunnel', 'city', 'geometry'],
      dtype='object')

In [7]:
delivery = pd.read_csv("./LaDe/delivery/delivery_hz.csv")

In [2]:
traj_df = pd.read_pickle("./data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")

# traj_df["gps_time"]=pd.to_datetime(traj_df["gps_time"])

traj_gsort_df = (
    traj_df.groupby("postman_id",group_keys=False)
    .apply(lambda g:g.sort_values("gps_time"))
    .reset_index(drop=True)
)



# traj_basegroup_df = (
#     traj_df.groupby(["ds","postman_id"])
#     .size()
#     .reset_index(name="count_coordinates")
# )



C:\Users\saran\AppData\Local\Temp\ipykernel_14296\430930721.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g:g.sort_values("gps_time"))


In [3]:
traj_gsort_df["seq"]=(
    traj_gsort_df.groupby("postman_id")[["lat","lng"]]
    .apply(lambda g: (g != g.shift())).any(axis=1).cumsum()
    .reset_index(level=0, drop=True)
)


In [4]:
traj_spacegroup_df = (
    traj_gsort_df.groupby(["postman_id","seq"])
    .agg(
        lat = ("lat","first"),
        lng = ("lng","first"),
        start_time = ("gps_time","min"),
        end_time = ("gps_time","max"),
        n_pings = ("gps_time","size")
    )
    .reset_index()
)

In [5]:
traj_spacegroup_df.to_pickle("trajectory_spacegroup.pkl")

In [7]:
delivery = pd.read_csv("./delivery/delivery_hz.csv")